# IBM Cloud SQL Query를 활용, COS에 저장된 화일을 처리할 수 있습니다.
<div class="pull-left"><left><img style="float: right;" src="http://developer.ibm.com/clouddataservices/wp-content/uploads/sites/85/2018/01/ibm-cloud-object-storage-logo-small.png" width="100" margin=50></left></div>
<div style="text-align:center">
<b>IBM Cloud SQL Query</b> is IBM's <b>serverless SQL</b> service on data in <b>Cloud Object Storage</b>. It allows to run ANSI SQL on Parquet, CSV and JSON data sets. It is based on Apache Spark SQL as the query engine in the background. This means you do <b>not</b> have to provision any Apache Spark instance or service. A simple Python client is sufficient.<br><br></div>
This notebook is meant to be a generic starter to use the SQL Query API in order to run SQL statements in a programmatic way. It uses the <a href="https://github.com/IBM-Cloud/sql-query-clients/tree/master/Python" target="_blank" rel="noopener noreferrer">ibmcloudsql</a> Python library for this purpose. The notebook also demonstrates how you can combine SQL Query with visualization libraries such as **PixieDust**. The notebook has been verified to work with Python 3.5. As mentioned above it does not require a Spark service bound to the notebook.

## Table of contents
1. [Setup libraries](#setup)<br>
2. [Configure SQL Query](#configure)<br>
    2.1 [Using the project bucket](#projectbucket)<br>
    2.2 [Setting SQL Query parameters](#parameters)<br>
3. [Your SQL](#sql)<br>
4. [Run the SQL](#run)<br>
    4.1 [Low level SQL job submission](#lowlevel)<br>
5. [List recent SQL submissions](#joblist)<br>
6. [Useful things](#other)<br>
7. [Next steps](#next)<br>

### - 몇가지 사전 준비 작업
*** 1. platform API Key 생성*** (https://github.com/moreal70/IBM-Watson-Studio/blob/master/HowTo-Create%20Platform%20API%20Key.ipynb)     
*** 2. project token insert ***
![1](https://github.com/moreal70/IBM-Watson-Studio/raw/master/images/project_token_insert.png)       
*** 3. IBM Cloud SQL Query에서 CRN 가져오기 ***  
![1](https://github.com/moreal70/IBM-Watson-Studio/raw/master/images/cloud_sql_query_CRN.png)       

### <a id="setup"></a> 1. Setup libraries

* ***-q 옵션을 주면 output 이 적게 또는 아예 안보입니다.***

In [17]:
!pip -q install ibmcloudsql
!pip -q install sqlparse

In [2]:
import ibmcloudsql
from pixiedust.display import *
import pandas as pd
targeturl=''

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


### <a id="configure"></a> 2. Configure SQL Query
1. You need an **API key** for an IBM cloud identity that has access to your Cloud Object Storage bucket for writing SQL results and to your SQL Query instance. To create API keys log on to the IBM Cloud console and go to <a href="https://console.bluemix.net/iam/#/apikeys" target="_blank">Manage->Security->Platform API Keys</a>, click the `Create` button, give the key a custom name and click `Create`. In the next dialog click `Show` and copy the key to your clipboard and paste it below in this notebook.
2. You need the **instance CRN** for the SQL Query instance. You can find it in the <a href="https://console.bluemix.net/dashboard/apps" target="_blank">IBM Cloud console dashboard</a>. Make sure you have `All Resources` selected as resource group. In the section `Services` you can see your instances of SQL Query and Cloud Object Storage. Select the instance of SQL Query that you want to use. In the SQL Query dashboard page that opens up you find a section titled **REST API** with a button labelled **Instance CRN**. Click the button to copy the CRN into your clipboard and paste it here into the notebook. If you don't have an SQL Query instance created yet, <a href="https://console.bluemix.net/catalog/services/sql-query" target="_blank">create one</a> first.
3. You need to specify the location on Cloud Object Storage where your **query results** should be written. This comprises three parts of information that you can find in the Cloud Object Storage UI for your instance in the IBM Cloud console. You need to provide it as a **URL** using the format `cos://<endpoint>/<bucket>/[<prefix>]`. You have the option to use the cloud object storage **bucket that is associated with your project**. In this case, execute the following section before you proceed.  
<br/>
For more background information, check out the SQL Query <a href="https://console.bluemix.net/docs/services/sql-query/getting-started.html#getting-started-tutorial" target="_blank">documentation</a>.

#### <a id="projectbucket"></a> 2.1 Using the project bucket
**Only** follow the instructions in this section when you want to write your SQL query results to the bucket that has been created for the project for which you have created this notebook. In any other case proceed directly with section **2.2**.
<br><br>
__Inserting the project token__:  
Click the `More` option in the toolbar above (the three stacked dots) and select `Insert project token`.
 * If you haven't created an access token for this project before, you will see a dialog that asks you to create one first. Follow the link to open your project settings, scroll down to `Access tokens` and click `New token`. Give the token a custom name and make sure you select `Editor` as `Access role for project`. After you created your access token you can come back to this notebook, select the empty cell below and again select `Insert project token` from the toolbar at the top.
[//]: # 
This will add a new cell at the top of your notebook with content that looks like this:
```
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='<some id>', project_access_token='<some access token>')
pc = project.project_context
```
Leave that cell content as inserted and run the cell. Then then proceed with the following cell below:

* ***project token 가져오기***

In [3]:
# The code was removed by DSX for sharing.

* ***project meta data 에서 cos bucket 정보를 추출합니다.***

In [18]:
cos_bucket = project.get_metadata()['entity']['storage']['properties']
targeturl="cos://" + cos_bucket['bucket_region'] + "/" + cos_bucket['bucket_name'] + "/"

In [ ]:
print(cos_bucket)

#### <a id="parameters"></a> 2.2 Setting the SQL Query parameters

In [5]:
import getpass
apikey=getpass.getpass('Enter IBM Cloud API Key (leave empty to use previous one): ') or apikey
instnacecrn=input('Enter SQL Query Instance CRN (leave empty to use previous one): ') or instnacecrn
if targeturl == '':
    targeturl=input('Enter target URL for SQL results: ')
else:
    targeturl=input('Enter target URL for SQL results (leave empty to use ' + targeturl + '): ') or targeturl
sqlClient = ibmcloudsql.SQLQuery(apikey, instnacecrn, targeturl, client_info='SQL Query Starter Notebook')
sqlClient.logon()
print('\nYour SQL Query web console link:\n')
sqlClient.sql_ui_link()

Enter IBM Cloud API Key (leave empty to use previous one): ········
Enter SQL Query Instance CRN (leave empty to use previous one): crn%3Av1%3Abluemix%3Apublic%3Asql-query%3Aus-south%3Aa%2F651c233bb1b6ff339e8628e3ccd5b96a%3A9646c9a5-6825-45af-a24d-39b04c8d98da%3A%3A
Enter target URL for SQL results (leave empty to use cos://us-geo/projectparkhsu02d1be16ce7e164fc694f1c69332955f0b/): 

Your SQL Query web console link:

https://sql.ng.bluemix.net/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/651c233bb1b6ff339e8628e3ccd5b96a:9646c9a5-6825-45af-a24d-39b04c8d98da::


* ***위에서 튀어 나온 url 로 들어가서 아래 sql 문장을 돌려봐도 좋습니다.***

### <a id="sql"></a> 3. Your SQL
To author your own SQL query, use the interactive SQL Query web console (**link above**) of your SQL Query service instance.

In [6]:
import sqlparse
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter, Terminal256Formatter

sql=input('Enter your SQL statement (leave empty to use a simple sample SQL)')
if sql == '':
    sql='SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > "1998-01-01" ORDER BY c.CompanyName'
formatted_sql = sqlparse.format(sql, reindent=True, indent_tabs=True, keyword_case='upper')
lexer = get_lexer_by_name("sql", stripall=True)
formatter = Terminal256Formatter(style='tango')
result = highlight(formatted_sql, lexer, formatter)
from IPython.core.display import display, HTML
print('\nYour SQL statement is:\n')
print(result)

Enter your SQL statement (leave empty to use a simple sample SQL)

Your SQL statement is:

SELECT o.OrderID,
	c.CompanyName,
	e.FirstName,
	e.LastName
FROM cos://us-geo/SQL/orders.parquet STORED AS PARQUET o,
	cos://us-geo/SQL/employees.parquet STORED AS PARQUET e,
	cos://us-geo/SQL/customers.parquet STORED AS PARQUET c
WHERE e.EmployeeID = o.EmployeeID
		AND c.CustomerID = o.CustomerID
		AND o.ShippedDate > o.RequiredDate
		AND o.OrderDate > "1998-01-01"
ORDER BY c.CompanyName



### <a id="run"></a> 4. Running the SQL
The following cell submits the SQL statement and waits for it to finish before printing a sample of the result set.

In [20]:
result_df = sqlClient.run_sql(sql)
if isinstance(result_df, str):
    print(result_df)

In [21]:
result_df.head(10)

,OrderID,CompanyName,FirstName,LastName
0,10924,Berglunds snabbköp,Janet,Leverling
1,11058,Blauer See Delikatessen,Anne,Dodsworth
2,10827,Bon app',Nancy,Davolio
3,11076,Bon app',Margaret,Peacock
4,11045,Bottom-Dollar Markets,Michael,Suyama
5,10970,Bólido Comidas preparadas,Anne,Dodsworth
6,11054,Cactus Comidas para llevar,Laura,Callahan
7,11008,Ernst Handel,Robert,King
8,11072,Ernst Handel,Margaret,Peacock
9,10816,Great Lakes Food Market,Margaret,Peacock


* ***앞에 url 에 들어가서 조회한 것과 동일한 결과가 나와야합니다***

In [9]:
from pixiedust.display import *
display(result_df)

#### <a id="lowlevel"></a> 4.1 Low level SQL job submission
Let's run the same SQL again, but this time using the asynchronous submission mechanism and the status check method.

In [10]:
sqlClient.logon()
jobId = sqlClient.submit_sql(sql)
print("SQL query submitted and running in the background. jobId = " + jobId)

SQL query submitted and running in the background. jobId = a94962de-f901-4a8a-bfe9-eeb73d81aa98


In [11]:
print("Job status for " + jobId + ": " + sqlClient.get_job(jobId)['status'])

Job status for a94962de-f901-4a8a-bfe9-eeb73d81aa98: running


Use the `wait_for_job()` method as a blocking call until your job has finished:

In [12]:
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
if job_status == 'failed':
    details = sqlClient.get_job(jobId)
    print("Error: {}\nError Message: {}".format(details['error'], details['error_message']))

Job a94962de-f901-4a8a-bfe9-eeb73d81aa98 terminated with status: completed


Use the `get_result()` method to retrieve a dataframe for the SQL result set:

In [13]:
result_df = sqlClient.get_result(jobId)
print("OK, we have a dataframe for the SQL result that has been stored by SQL Query in " + sqlClient.get_job(jobId)['resultset_location'])

OK, we have a dataframe for the SQL result that has been stored by SQL Query in cos://s3-api.us-geo.objectstorage.softlayer.net/projectparkhsu02d1be16ce7e164fc694f1c69332955f0b/jobid=a94962de-f901-4a8a-bfe9-eeb73d81aa98


You can delete the result set from Cloud Object Storage using the `delete_result()` method:

In [14]:
sqlClient.delete_result(jobId)

,Deleted Object
0,jobid=a94962de-f901-4a8a-bfe9-eeb73d81aa98/_SU...
1,jobid=a94962de-f901-4a8a-bfe9-eeb73d81aa98
2,jobid=a94962de-f901-4a8a-bfe9-eeb73d81aa98/par...


### <a id="joblist"></a> 5. Listing recent SQL submissions
The method `get_jobs()` provides you a dataframe with all recent SQL submissions with all details. You can change the value `-1`for `display.max_colwidth` to a positive integer if you want to truncate the cell content to shrink the overall table display size.

In [15]:
pd.set_option('display.max_colwidth', -1)
sqlClient.get_jobs().head(100)

,job_id,status,user_id,statement,resultset_location,submit_time,end_time,error,error_message
0,a94962de-f901-4a8a-bfe9-eeb73d81aa98,completed,parkhsu@kr.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName",cos://s3-api.us-geo.objectstorage.softlayer.net/projectparkhsu02d1be16ce7e164fc694f1c69332955f0b/jobid=a94962de-f901-4a8a-bfe9-eeb73d81aa98,2018-05-01T02:28:42.440Z,2018-05-01T02:29:09.630Z,None,None
1,bcc6a50d-2fca-4ad1-8e08-05eacf9553bb,completed,parkhsu@kr.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName",cos://s3-api.us-geo.objectstorage.softlayer.net/projectparkhsu02d1be16ce7e164fc694f1c69332955f0b/jobid=bcc6a50d-2fca-4ad1-8e08-05eacf9553bb,2018-05-01T02:27:05.181Z,2018-05-01T02:27:32.212Z,None,None


### <a id="other"></a> 6. Useful things
`ibmcloudsql` provides some other methods that can be quite handy when working with data on IBM Cloud Object Storage. One is `get_cos_summary()` that provides statistics for a given **URL** to object storage. So you can for instance check the amount of objects and data volume that will be processed by your SQL query when you use the URL in it. For demonstration purposes we will now simply get statistics for the `targeturl` that we used above as target for our query results.

In [16]:
sqlClient.get_cos_summary(targeturl)

{'largest_object': 'notebook/03-unsupervised-learning_Ny5xlr35V.ipynb',
 'largest_object_size': '25.3 MB',
 'newest_object_timestamp': 'May 01, 2018, 02H:28M:43S',
 'oldest_object_timestamp': 'April 02, 2018, 09H:27M:38S',
 'smallest_object': 'jobid=bcc6a50d-2fca-4ad1-8e08-05eacf9553bb',
 'smallest_object_size': '0.0 B',
 'total_objects': 69,
 'total_volume': '34.1 MB',
 'url': 'cos://us-geo/projectparkhsu02d1be16ce7e164fc694f1c69332955f0b/'}

### <a id="next"></a> 7. Next steps
In this notebook you learned how you can use the `ibmcloudsql` library in a Python notebook to submit SQL queries on data in IBM Cloud Object Storage and how you can interact with the query results. If you want to automate such an SQL query execution as part of your cloud solution, you can use the <a href="https://console.bluemix.net/openwhisk/" target="_blank">IBM Cloud Functions</a> framework. There is a dedicated SQL function available that lets you set up a cloud function to run SQL statements with IBM Cloud SQL Query. You can find the documentation for doing this <a href="https://hub.docker.com/r/ibmfunctions/sqlquery/" target="_blank" rel="noopener noreferrer">here</a>.

### <a id="authors"></a>Authors

**Torsten Steinbach**, Torsten is the lead architect for IBM Cloud SQL Query. Previously he has worked as IBM architect for a series of data management products and services, including DB2, PureData for Analytics and Db2 on Cloud.

<hr>
Copyright &copy; IBM Corp. 2018. This notebook and its source code are released under the terms of the MIT License.